# Create Azure SQL Virtual Machine

## Description

For more information about other Azure PowerShell options for creating SQL VMs, see the [Provisioning guide for SQL Server VMs with Azure PowerShell](https://docs.microsoft.com/en-us/azure/virtual-machines/windows/sql/virtual-machines-windows-ps-sql-create). See also [Quickstart guide](https://docs.microsoft.com/en-us/azure/virtual-machines/windows/sql/quickstart-sql-vm-create-powershell) to creating a SQL Server VM with Azure PowerShell.

Steps of this procedure include:

1. Define notebook variables
2. Connect to Azure subscription
3. Provision resource group for SQL VM migration
4. Create a storage account
5. Configure Network Settings
6. Provision SQL VM
7. Configure SQL VM IaaS agent

## Define Notebook Variables

In [ ]:
# Resource Group  
$Subscription       = "" # Leave blank to open a dialog to get Azure Subscription, or explicitly specify
$ResourceGroup      = "" # Name of the resource group to create in the current subscription
$Location           = "" # see Appendix for a list of location settings

# Compute
$VMName             = "" # VM to create
$PublisherName      = "" # Name of Publisher, Default would be 'MicrosoftSQLServer'
$Version            = "" # Version of VM, Default would be 'latest'

# Storage
$StorageAccountName =   $ResourceGroup + "_storage"
$StorageSKU         = "" # Choose your storage SKU (see appendix)
$StorageName        = "sqlstorage" + (Get-Random -Minimum 1 -Maximum 100)

# VM Password
$secureVMPassword   = "" # Create the password  for VM.

## Get Subscription

Open a dialog box with a list of subscriptions, if one isn't specified. Selecting one will set that subscription in the Az CLI context for rest of the notebook.

In [ ]:
if (!$Subscription)
{
    $Subscription = Get-AzSubscription | Out-GridView -PassThru
} 
Set-AzContext -SubscriptionName $Subscription
Connect-AzAccount -Subscription $Subscription

## Create Azure Resource Group

In [ ]:
# Create Azure resource group, if necessary
$rg = Get-AzResourceGroup | Where ResourceGroupName -eq $ResourceGroup

if (!$rg)
{
    # Need to create a new resource group
    Write-Output "Resource Group '$ResourceGroup' does not exist"
    $rg = New-AzResourceGroup -Name $ResourceGroup -Location $Location
}

## Create a storage account
VMs require storage resources for OS, SQL data and logs. Create a new storage account as a place for it.

In [ ]:
$StorageAccount = Get-AzStorageAccount | Where StorageAccountName -eq $StorageAccountName

if (!$StorageAccount)
{
    Write-Output "Storage Account $StorageName does not exist. Creating..."
    $StorageAccount = New-AzStorageAccount -ResourceGroupName $ResourceGroup -Name $StorageName -SkuName $StorageSku -Kind "Storage" -Location $Location
}

## Configure network settings
Create a virtual network, subnet, and a public IP address. These resources are used to provide network connectivity to the virtual machine and connect it to the internet.

In [ ]:
$SubnetName = $ResourceGroup + "subnet"
$VnetName = $ResourceGroup + "vnet"
$PipName = $ResourceGroup + $(Get-Random)

# Create a subnet configuration
$SubnetConfig = New-AzVirtualNetworkSubnetConfig -Name $SubnetName -AddressPrefix 192.168.1.0/24

# Create a virtual network
$Vnet = New-AzVirtualNetwork -ResourceGroupName $ResourceGroup -Location $Location `
   -Name $VnetName -AddressPrefix 192.168.0.0/16 -Subnet $SubnetConfig

# Create a public IP address and specify a DNS name
$Pip = New-AzPublicIpAddress -ResourceGroupName $ResourceGroup -Location $Location `
   -AllocationMethod Static -IdleTimeoutInMinutes 4 -Name $PipName

2. Create a network security group. Configure rules to allow remote desktop (RDP) and SQL Server connections.

In [ ]:
# Rule to allow remote desktop (RDP)
$NsgRuleRDP = New-AzNetworkSecurityRuleConfig -Name "RDPRule" -Protocol Tcp `
   -Direction Inbound -Priority 1000 -SourceAddressPrefix * -SourcePortRange * `
   -DestinationAddressPrefix * -DestinationPortRange 3389 -Access Allow

#Rule to allow SQL Server connections on port 1433
$NsgRuleSQL = New-AzNetworkSecurityRuleConfig -Name "MSSQLRule"  -Protocol Tcp `
   -Direction Inbound -Priority 1001 -SourceAddressPrefix * -SourcePortRange * `
   -DestinationAddressPrefix * -DestinationPortRange 1433 -Access Allow

# Create the network security group
$NsgName = $ResourceGroup + "nsg"
$Nsg = New-AzNetworkSecurityGroup -ResourceGroupName $ResourceGroup `
   -Location $Location -Name $NsgName `
   -SecurityRules $NsgRuleRDP,$NsgRuleSQL

3. Create the network interface.

In [ ]:
$InterfaceName = $ResourceGroup + "int"
$Interface = New-AzNetworkInterface -Name $InterfaceName `
   -ResourceGroupName $ResourceGroup -Location $Location `
   -SubnetId $VNet.Subnets[0].Id -PublicIpAddressId $Pip.Id `
   -NetworkSecurityGroupId $Nsg.Id

## Create the SQL VM
1. Define your credentials to sign in to the VM. The username is "azureadmin". Make sure you change <password> before running the command.

In [ ]:
# Define a credential object
$SecurePassword = ConvertTo-SecureString $secureVMPassword `
   -AsPlainText -Force
$Cred = New-Object System.Management.Automation.PSCredential ("azureadmin", $securePassword)

2. Create a virtual machine configuration object and then create the VM. The following command creates a SQL Server 2017 Developer Edition VM on Windows Server 2016.

In [ ]:
# Create a virtual machine configuration

$VMConfig = New-AzVMConfig -VMName $VMName -VMSize Standard_DS13_V2 |
   Set-AzVMOperatingSystem -Windows -ComputerName $VMName -Credential $Cred -ProvisionVMAgent -EnableAutoUpdate |
   Set-AzVMSourceImage -PublisherName "MicrosoftSQLServer" -Offer "SQL2017-WS2016" -Skus "SQLDEV" -Version "latest" |
   Add-AzVMNetworkInterface -Id $Interface.Id

## Create Virtual Machine

In [ ]:
# Create the VM
New-AzVM -ResourceGroupName $ResourceGroup -Location $Location -VM $VMConfig

## Install the SQL IaaS Agent
To get portal integration and SQL VM features, you must install the SQL Server IaaS Agent Extension. To install the agent on the new VM, run the following command after the VM is created.

In [ ]:
Set-AzVMSqlServerExtension -ResourceGroupName $ResourceGroup -VMName $VMName -name "SQLIaasExtension" -version "2.0" -Location $Location